In [44]:
PROJECT_ID = 'your-project-id'
bucket_name = 'your-gcs-bucket'

In [45]:
import base64
import vertexai
from vertexai.preview.generative_models import *
from google.cloud import storage
from PIL import Image
import pandas as pd


In [46]:
client = storage.Client()
bucket = client.get_bucket(bucket_name)

In [47]:
def generate_text(b64_image: str, prompt: str, mimeType: str) -> str:

    # Create a GenerativeModel object for the specified model
    multimodal_model = GenerativeModel("gemini-pro-vision")

    # Generate content using the model, passing the image and prompt
    response = multimodal_model.generate_content(
        [
            Part.from_data(
                data=b64_image, mime_type=mimeType # Use the mime_type variable
            ),
            Part.from_text(prompt),  # Encapsulate the prompt in a Part object
        ]
    )
    return response.text

In [49]:
def open_image(image_bytes):
    encoded_base_image = base64.b64encode(image_bytes)
    B64_BASE_IMAGE = encoded_base_image.decode('utf-8')
    return B64_BASE_IMAGE

suffixes = (".jpg", ".jpeg")









In [ ]:
def make_magic(prompt):
    responses = []
    file_names = []
    mimeType = "image/jpeg"
    blobs = bucket.list_blobs(prefix='raw_images/')
    for blob in blobs:
        if blob.name.endswith(suffixes):
            image_bytes = blob.download_as_bytes()
            image = open_image(image_bytes)
            response = generate_text(image, prompt, mimeType)
            #print('image_file_is:', blob)
            #print(response)
            responses.append(response)
            file_names.append(blob.name)
    return responses, file_names

# Set mime type based on media type selection
#mimeType = "image/jpeg"

#prompt_text = "Please identify the products in this image and return results in json format with Brand as an attribute"

# Generate text using Vertex AI, passing the mime type argument
#generated_text = generate_text(B64_BASE_IMAGE, prompt_text, mimeType)

In [ ]:
prompt_text = "Please identify the products in this image"
responses, file_names = make_magic(prompt_text)

In [36]:
response_df = pd.DataFrame(responses, columns=['llm_response'])

In [37]:
file_names_df = pd.DataFrame(file_names, columns=['image_name'])

In [41]:
final_df = file_names_df.join(response_df, how='inner')

In [43]:
final_df.to_csv('final_output.csv')
